# Model

### package load

In [1]:
import datetime
import numpy as np
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys
import time
from tqdm import tqdm
import numpy as np
import warnings
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms.functional as TF
import torch.nn.functional as F
from torch.autograd import Variable

from model_new import convert_bn_to_instancenorm, convert_bn_to_evonorm, convert_bn_to_groupnorm, DeepLabHead 
from helpers import AverageMeter, ProgressMeter, iouCalc, visim, vislbl
from labels import labels

### device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
#device = 'cpu'

### data load

In [4]:
npzfile = np.load('data_norm_v2.npz') ## or 'data_origin.npz' 원본 or norm ver :: 모델 결과 확인 후 결정

train_x = npzfile['train_x']
train_y = npzfile['train_y']
val_x = npzfile['val_x']
val_y = npzfile['val_y']
test_x = npzfile['test_x']
test_y = npzfile['test_y']

npzfile.close()

### DeepLab v3 ResNet50 

In [5]:
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False).to(device)
         # dlv3 backbone resnoet 50 :: output chnnel: 2048 --> DL input channel :: 2048
model.classifier = DeepLabHead(2048, 12).to(device) # 12 = class num
   # 작동되는 지 확인 필요 (개수 맞추기)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=2) 

In [6]:
# Initialize metrics
best_miou = 0.0
metrics = {'train_loss' : [],
           'train_acc' : [],
           'test_acc' : [],
           'test_loss' : [],
           'miou' : []}
start_epoch = 0

### Label 

In [7]:
# Create list of class names
  # classLabels: label명
  # validClasses: label에 해당하는 id값

classLabels = []
for label in labels:
    if label.name not in classLabels:
        classLabels.append(label.name)
classLabels.append('void')

validClasses = list(
    np.unique(
        [label.id for label in labels 
         if label.id >= 0
        ] + [11]
    )
)

### train

In [8]:
## parameter

batch_size = 2
num_epoch = 50

result_dir = os.getcwd() + '/result/' + 'new/'

In [9]:
from torch.utils.data import TensorDataset, DataLoader

   # X: 0~255, RGB값 가지는 이미지 데이터 (560, 720, 960, 3)
   # Y: 0~11 label값 가지는 픽셀당 클래스 데이터 (560, 720, 960)
X = torch.tensor(train_x, dtype=torch.float32)
Y = torch.tensor(train_y, dtype=torch.long)

data = TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)
train_data = DataLoader(data, batch_size=batch_size, shuffle=True)

In [10]:
# Create class weight
label_num = {str(_id): 0 for _id in validClasses}
label_num

for y in train_y.flatten():
    label_num[str(y)] += 1
label_num

{'0': 3350644,
 '1': 3682379,
 '2': 1888603,
 '3': 2478108,
 '4': 81471634,
 '5': 4996800,
 '6': 23813698,
 '7': 98899475,
 '8': 16468486,
 '9': 53441175,
 '10': 37943059,
 '11': 10253939}

In [11]:
# delete void -- max value
max_num = max([v for k, v in label_num.items() if k != '11'])
weights = [max_num/num for key, num in label_num.items()]

class_weights = torch.FloatTensor(weights).to(device)
criterion = nn.CrossEntropyLoss(weight = class_weights, ignore_index=12) # weight 파라미터에 class_weight 추가

In [12]:
class_weights

tensor([29.5166, 26.8575, 52.3665, 39.9093,  1.2139, 19.7926,  4.1530,  1.0000,
         6.0054,  1.8506,  2.6065,  9.6450])

In [ ]:
model.train()
res = X.shape[1] * X.shape[2] # 720 * 960

for epoch in range(num_epoch):
    
    loss_running = AverageMeter('Loss', ':.4e')
    acc_running = AverageMeter('Accuracy', ':.3f')
    
    iou = iouCalc(classLabels, validClasses, voidClass = 11)
    progress = ProgressMeter(
        len(train_data),
        [loss_running, acc_running],
        prefix="Train, epoch: [{}]".format(epoch))
    
    batch_loss = 0.0
    for batch, (x, y) in enumerate(tqdm(train_data, total=len(train_data))):
        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(x)
        outputs = outputs['out']
        preds = torch.argmax(outputs, 1)
        
        # cross-entropy loss
        loss = criterion(outputs, y)

        # backward pass
        loss.backward()
        optimizer.step()
        
        # Statistics
        bs = x.size(0)
        loss = loss.item()
        loss_running.update(loss, bs)
        corrects = torch.sum((preds == y) & (y != 12))
        
        nvoid = int((y==12).sum()) 
        acc = corrects.double()/(bs*res-nvoid)
        acc_running.update(acc, bs)
        
        # Calculate IoU scores of current batch
        iou.evaluateBatch(preds, y)
        
        progress.display(epoch)
        
        
    scheduler.step(loss_running.avg)
    miou = iou.outputScores()
    
    # save checkpoint
    now = datetime.datetime.now()
    now_time = now.strftime('%y%m%d%H')
    
    # save path
    if not os.path.isdir(result_dir):
        os.makedirs(result_dir)
    
    save_path = result_dir
    
    with open(save_path + 'train_logs.csv', 'a') as epochs:
            epochs.write('{}, {:.4f}, {:.4f}, {:.4f}\n'.format(
                    epoch+1, loss_running.avg, acc_running.avg, miou))
            
    # Save model to file
    torch.save({
        'epoch' : epoch + 1,
        'model_state_dict' : model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'best_miou': best_miou,
        'metrics': metrics,
        }, save_path + now_time + '_E' + str(epoch+1) + '_checkpoint.pth.tar')
    
    # Save best model to file
    if miou > best_miou:
        print('mIoU improved from {:.4f} to {:.4f}.'.format(best_miou, miou))
        best_miou = miou
        
    print('epoch ', epoch)
    print('loss : {:.4f}   acc : {:.4f}   miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))

  0%|▎                                                                               | 1/245 [00:17<1:10:18, 17.29s/it]

Train, epoch: [0][  0/245]	Loss 2.5794e+00 (2.5794e+00)	Accuracy 0.104 (0.104)


  1%|▋                                                                               | 2/245 [00:34<1:09:18, 17.11s/it]

Train, epoch: [0][  0/245]	Loss 2.4892e+00 (2.5343e+00)	Accuracy 0.109 (0.107)


  1%|▉                                                                               | 3/245 [00:51<1:10:04, 17.37s/it]

...

Train, epoch: [0][  0/245]	Loss 2.5302e+00 (1.9962e+00)	Accuracy 0.572 (0.310)


  7%|█████▊                                                                         | 18/245 [05:17<1:07:15, 17.78s/it]

Train, epoch: [0][  0/245]	Loss 1.2788e+00 (1.9563e+00)	Accuracy 0.615 (0.326)


  8%|██████▏                                                                        | 19/245 [05:34<1:06:35, 17.68s/it]

Train, epoch: [0][  0/245]	Loss 1.5987e+00 (1.9375e+00)	Accuracy 0.605 (0.341)


### val

In [ ]:
X = torch.tensor(val_x, dtype=torch.float32)
Y = torch.tensor(val_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

val_data = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

save_path = result_dir
result = sorted(os.listdir(save_path), reverse=True)

In [ ]:
result

In [ ]:
for epoch, file in enumerate(result[1:]):
    checkpoint = torch.load(save_path + file)
    model.load_state_dict(checkpoint['model_state_dict'], strict=True)
    
    model.eval()
    
    loss_running = AverageMeter('Loss', ':.4e')
    acc_running = AverageMeter('Accuracy', ':.3f')
    
    iou = iouCalc(classLabels, validClasses, voidClass = 11)
    
    batch_loss = 0.0
    
    for batch, (x, y) in enumerate(tqdm(val_data, total=len(val_data))):

        x = x.to(device)
        y = y.to(device)

        # forward
        outputs = model(x)
        outputs = outputs['out']

        preds = torch.argmax(outputs, 1)

        # cross-entropy loss
        loss = criterion(outputs, y)

        # Statistics
        bs = x.size(0)
        loss = loss.item()
        loss_running.update(loss, bs)
        corrects = torch.sum((preds == y) & (y != 12))

        nvoid = int((y==12).sum())
        acc = corrects.double()/(bs*res-nvoid)
        acc_running.update(acc, bs)
    
        # Calculate IoU scores of current batch
        iou.evaluateBatch(preds, y)

    miou = iou.outputScores()
    
    with open(save_path + 'val_logs.csv', 'a') as epochs:
            epochs.write('{}, {:.4f}, {:.4f}, {:.4f}\n'.format(
                    epoch+1, loss_running.avg, acc_running.avg, miou))
    
    print('val- epoch: {}'.format(epoch+1))
    print('loss : {:.4f} acc : {:.4f} miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou)) 

### plot

In [ ]:
train_logs = pd.read_csv(save_path + 'train_logs.csv', names = ['epoch', 'loss', 'accuracy', 'miou'])
val_logs = pd.read_csv(save_path + 'val_logs.csv', names = ['epoch', 'loss', 'accuracy', 'miou'])

plt.figure(figsize=(15, 5))
plt.plot(train_logs['epoch'], train_logs['miou'], label = 'train_miou')
plt.plot(train_logs['epoch'], train_logs['accuracy'], label = 'train_accuracy')

plt.plot(val_logs['epoch'], val_logs['miou'], label = 'val_miou')
plt.plot(val_logs['epoch'], val_logs['accuracy'], label = 'val_accuracy')

plt.xticks([i for i in range(1, train_logs.shape[0]+1)])
plt.title("Performances of train, val dataset")
plt.xlabel("epoch")
plt.ylabel("")
plt.legend()
plt.show()

### test

In [16]:
X = torch.tensor(test_x, dtype=torch.float32)
Y = torch.tensor(test_y, dtype=torch.long)

data = torch.utils.data.TensorDataset(X.permute(dims=(0, 3, 1, 2)), Y)

test_data = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)

#save_path = result_dir
#result = sorted(os.listdir(save_path), reverse=True)

In [46]:
result

['0605_best_weights.pth.tar']

In [19]:
checkpoint = torch.load(save_path + result[2]) # train, val 결과보고 결정 (overfitting 시작되기 전)
model.load_state_dict(checkpoint['model_state_dict'], strict=True)

<All keys matched successfully>

In [ ]:
batch_time = AverageMeter('Time', ':6.3f')
data_time = AverageMeter('Data', ':6.3f')
progress = ProgressMeter(
    len(test_data),
    [batch_time, data_time],
    prefix='Predict: ')
loss_running = AverageMeter('Loss', ':.4e')
acc_running = AverageMeter('Accuracy', ':.3f')
    
iou = iouCalc(classLabels, validClasses, voidClass = 11)

model.eval()

batch_loss = 0.0
for batch, (x, y) in enumerate(tqdm(test_data, total=len(test_data))):

    x = x.to(device)
    y = y.to(device)

    # forward
    outputs = model(x)
    outputs = outputs['out']

    preds = torch.argmax(outputs, 1)

    # cross-entropy loss
    loss = criterion(outputs, y)

    # Statistics
    bs = x.size(0)
    loss = loss.item()
    loss_running.update(loss, bs)
    corrects = torch.sum((preds == y) & (y != 12))

    nvoid = int((y==12).sum())
    acc = corrects.double()/(bs*res-nvoid)
    acc_running.update(acc, bs)

    # Calculate IoU scores of current batch
    iou.evaluateBatch(preds, y)

miou = iou.outputScores()

print('loss : {:.4f} acc : {:.4f} miou : {:.4f}'.format(loss_running.avg, acc_running.avg, miou))